# 6. Finale Demo Applicatie

Dit notebook bevat de interactieve demo van ons schilderijen-classificatiemodel.
We gebruiken de library **Gradio** om een web-interface te maken waar gebruikers een eigen afbeelding kunnen uploaden.

Het model dat we gebruiken is **VGG16 met Transfer Learning**, getraind op de Vlaamse Supercomputer (VSC), met een nauwkeurigheid van ~94%.

## Imports en model bouwen

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
import gradio as gr
import numpy as np
from PIL import Image
import os
import h5py

# 1. Instellingen
IMG_SIZE = (224, 224)
CLASS_NAMES = ["Mondriaan", "Picasso", "Rembrandt", "Rubens"] # Alfabetisch!
MODEL_PATH = "../models/transfer_model_vgg16.h5"

# 2. Model opbouwen (exacte architectuur als training)
print("Model opbouwen...")

conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
conv_base.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = layers.RandomFlip("horizontal")(inputs)
x = layers.RandomRotation(0.1)(x)
x = layers.RandomZoom(0.1)(x)
x = tf.keras.applications.vgg16.preprocess_input(x)
x = conv_base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu', name='dense')(x)
x = layers.Dropout(0.5, name='dropout')(x)
outputs = layers.Dense(4, activation='softmax', name='dense_1')(x)

model = models.Model(inputs, outputs)

# 3. Gewichten laden uit het .h5 bestand
print(f"\nGewichten laden van {MODEL_PATH}...")

with h5py.File(MODEL_PATH, 'r') as f:
    # De dense lagen bevatten onze getrainde gewichten
    # Structuur: model_weights/layer_name/layer_name/kernel:0 en bias:0
    
    # Dense laag (256 units)
    dense_kernel = f['model_weights/dense/dense/kernel:0'][:]
    dense_bias = f['model_weights/dense/dense/bias:0'][:]
    
    # Dense_1 laag (4 units - output)
    dense1_kernel = f['model_weights/dense_1/dense_1/kernel:0'][:]
    dense1_bias = f['model_weights/dense_1/dense_1/bias:0'][:]
    
    print(f"Dense kernel shape: {dense_kernel.shape}, bias: {dense_bias.shape}")
    print(f"Dense_1 kernel shape: {dense1_kernel.shape}, bias: {dense1_bias.shape}")

# Gewichten toepassen op ons model
for layer in model.layers:
    if layer.name == 'dense':
        layer.set_weights([dense_kernel, dense_bias])
        print("✅ Dense laag gewichten geladen")
    elif layer.name == 'dense_1':
        layer.set_weights([dense1_kernel, dense1_bias])
        print("✅ Dense_1 (output) laag gewichten geladen")

print("\n✅ Model klaar voor gebruik!")

Structuur van het opgeslagen .h5 bestand:

Layer groups:
  - dense
  - dense_1
  - dropout
  - global_average_pooling2d
  - input_2
  - random_flip
  - random_rotation
  - random_zoom
  - tf.__operators__.getitem
  - tf.nn.bias_add
  - top_level_model_weights
  - vgg16


## De voorspelfunctie

Hier maken we de functie die Gradio gaat gebruiken

In [8]:
def classify_image(image):
    if image is None:
        return None
    
    # 1. Afbeelding voorbereiden
    # Gradio geeft een numpy array, wij willen een PIL image om te resizen
    img = Image.fromarray(image.astype('uint8'), 'RGB')
    img = img.resize(IMG_SIZE)
    
    # Omzetten naar array en batch dimensie toevoegen (1, 224, 224, 3)
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    
    # 2. Voorspellen
    prediction = model.predict(img_array, verbose=0)
    
    # 3. Resultaat omzetten naar dictionary {Label: Kans}
    # prediction[0] bevat de 4 kansen (bijv. [0.01, 0.90, 0.05, 0.04])
    result_dict = {CLASS_NAMES[i]: float(prediction[0][i]) for i in range(len(CLASS_NAMES))}
    
    return result_dict

## De interface starten

In [9]:
# Interface bouwen
interface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(label="Upload een schilderij"),
    outputs=gr.Label(num_top_classes=4, label="Voorspelling"),
    title="🎨 Schilderijen Herkenner AI",
    description="Upload een foto van een schilderij en het model vertelt of het een Mondriaan, Picasso, Rembrandt of Rubens is.",
    theme="default"
)

# Start de app
interface.launch(share=True)

/media/mathieu/Intenso/DL_Schilderijen/Deep_Learning_Schilderijen/.venv/lib/python3.13/site-packages/gradio/interface.py:171: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  super().__init__(


* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://449993694d69e452d8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
